In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [ ]:
import dimers as dim

In [ ]:
import KagomeFunctions as lattice

In [ ]:
import itertools

In [ ]:
from scipy.special import erfc

In [ ]:
from time import time

In [ ]:
import DualwormFunctions as dw
import Observables as obs

%load_ext line_profiler
%load_ext memory_profiler

import RunBasis
import argparse
import StartStates as strst
import numpy as np

import AnalysisFunctions as af # functions to make the analysis code cleaner
import KagomeDrawing as kdraw


import matplotlib.pyplot as plt
%matplotlib inline

import yep

In [ ]:
nst = 2048;
nsm = 16;
nips = 4;
nrps = 4;
nb = 4;
measperiod = 1;
nt = 128
nh = 1
ncores = 4
nmaxiter = 1
L = 8
J1 = -0.2
J2 = 1
J3 = 1
h = 0
notfullstateupdate = True
stroutput = "Phase8_J1-0.2J21J31_L{0}_tests_11".format(L)#_folder

In [ ]:
# Create a folder to save the pictures
foldername = "./" + stroutput + "_folder/" 
results_foldername = 'Results/'
os.makedirs(foldername + results_foldername, exist_ok = True)

filenamelist = ["backup"]
n = len(filenamelist)

In [ ]:
[L, numsites, J1, J2, J3, J3st, J4, nb, num_in_bin, temperatures, nt,
 stat_temps, temperatures_plots, hfields, nh,
 stat_hfields, hfields_plots, listfunctions, sref,ids2walker] =\
af.LoadParameters(foldername, filenamelist)

In [ ]:
kw = {'binning': False, 'plzplot': True, 'plotmin': 0, 'plotmax': 16}
[t_h_MeanE, t_h_MeanEsq, t_h_varMeanE, t_h_varMeanEsq, C, ErrC] = \
 af.LoadEnergy(foldername, filenamelist, numsites,
               nb, stat_temps, temperatures, stat_hfields,
               listfunctions, **kw)

In [ ]:
tidmin = 0
tidmax = [len(stat_temps[i]) for i in range(n)]

af.BasicPlotsE(L, n, tidmin, tidmax, temperatures_plots, hfields_plots, foldername,
                results_foldername, filenamelist, t_h_MeanE, t_h_MeanEsq, t_h_varMeanE,
                t_h_varMeanEsq, C, ErrC, J1, J2, J3, J4, S0 = np.log(2))

In [ ]:
# Load the states
t_spinstates, t_states, t_charges = af.LoadStates(stroutput+"_folder/", ["backup"], L, nh)
latsize = t_spinstates[0][0].size
[d_ijl, ijl_d, s_ijl, ijl_s, d_2s, s2_d, d_nd, d_vd, d_wn,
     sidlist, didlist, c_ijl, ijl_c, c2s, csign] =\
    dw.latticeinit(L[0])
print(ids2walker[0][1,0])

In [ ]:
[groundspinstates, groundstates, groundcharges] = \
 af.LoadGroundStates(stroutput+"_folder/", filenamelist, L, nh, [nsm],**kw)

groundspinstates = groundspinstates[0]
groundstates = groundstates[0]
groundcharges = groundcharges[0]

stlen = len(s_ijl)
domainwalls = obs.dweasyphase(stlen, groundstates[0], [], [], [],[])

In [ ]:

couplings = {'J1':J1[0], 'J2':J2[0],'J3':J3[0], 'J3st':J3[0]}
hamiltonian = dw.Hamiltonian(couplings, d_ijl, ijl_d, L[0])


In [ ]:
#### State analysis
for i in range(2):
    #state = t_states[0][ids2walker[0][i,0]]
    state = groundstates[i]
    #spinstate = t_spinstates[0][ids2walker[0][i,0]]
    spinstate = groundspinstates[i]
    #charges = obs.charges(0, state, 0 , spinstate, s_ijl, ijl_s,c2s = c2s , csign= csign)
    charges = groundcharges[i]
    print("Energy per site:" ,dw.compute_energy(hamiltonian, groundstates[i], len(s_ijl)))
    
    kdraw.plotstate(0, L[0], d_ijl, ijl_s, sidlist, didlist, s2_d, [state], [spinstate],
                    dim_node_color = "pink", dim_color =np.array([[1, 1, 0, 0]]), 
                    no_dim_color="darkcyan",
                    spin_up_color = 'blue', spin_down_color = 'red', dimerlinewidth=20)
    #kdraw.plot_honeycomb_chargestate(charges, ijl_c,
    #                                 L,2,c1_color = 'blue', c2_color = 'green',
    #                                 c3_color = 'red', c4_color = 'yellow', uponly=False)
    domainwalls = obs.dweasyphase(stlen, groundstates[i], [], [], [],[])
    print(sum(domainwalls))
    domains = [d for d, dstate in enumerate(state) if dstate ==-1 and domainwalls[d//6]==1]
    kdraw.plot_dice_dimers([d for d, dstate in enumerate(state) if dstate ==-1], d_ijl, L[0], 2,
                           color = 'darkcyan', linewidth=1, label='spin up - spin down separation', PBC= False)
    kdraw.plot_dice_dimers(domains, d_ijl, L[0], 2, color = 'darkcyan', linewidth=5, 
                           label='domain wall between mappings', PBC = False)
    plt.legend()
    plt.savefig("./"+stroutput+"_folder/PlotsResults/GSConfig{0}_redlines.png".format(i))
    
    
    #kdraw.plotstate(0, L, d_ijl, ijl_s, sidlist, didlist, s2_d, [state], [spinstate],
    #                dim_node_color = 'peachpuff', dim_color ='peachpuff', no_dim_color="red",
    #                spin_up_color = 'blue', spin_down_color = 'red')
    ##kdraw.plot_honeycomb_chargestate(charges, ijl_c,
    ##                                 L,2,c1_color = 'blue', c2_color = 'green',
    ##                                 c3_color = 'red', c4_color = 'yellow', uponly=False)
    #plt.legend()
    #plt.savefig("./"+stroutput+"_folder/PlotsResults/GSConfig{0}_redlines+dimers.png".format(i))

In [ ]:
#### State analysis
for i in range(2):
    #state = t_states[0][ids2walker[0][i,0]]
    state = groundstates[i]
    #spinstate = t_spinstates[0][ids2walker[0][i,0]]
    spinstate = groundspinstates[i]
    #charges = obs.charges(0, state, 0 , spinstate, s_ijl, ijl_s,c2s = c2s , csign= csign)
    charges = groundcharges[i]
    print("Energy per site:" ,dw.compute_energy(hamiltonian, groundstates[i], len(s_ijl)))
    
    kdraw.plotstate(0, L[0], d_ijl, ijl_s, sidlist, didlist, s2_d, [state], [spinstate],
                    dim_node_color = np.array([[1, 1, 0, 0]]), dim_color = np.array([[1, 1, 0, 0]]), 
                    no_dim_color="crimson", spin_edge_color='lightgrey',
                    spin_up_color = 'tan', spin_down_color = 'navy', PBC = False)
    #kdraw.plot_honeycomb_chargestate(charges, ijl_c,
    #                                 L,2,c1_color = 'blue', c2_color = 'green',
    #                                 c3_color = 'red', c4_color = 'yellow', uponly=False)
    domainwalls = obs.dweasyphase(stlen, groundstates[i], [], [], [],[])
    print(sum(domainwalls))
    domains = [d for d, dstate in enumerate(state) if dstate ==-1 and domainwalls[d//6]==1]
    kdraw.plot_dice_dimers([d for d, dstate in enumerate(state) if dstate ==-1], d_ijl, L[0], 2,
                           color = 'crimson', linewidth=1, PBC = False)
    kdraw.plot_dice_dimers(domains, d_ijl, L[0], 2, color = 'crimson', linewidth=6, PBC = False)
    #plt.legend()
    plt.savefig("./"+stroutput+"_folder/PlotsResults/GSConfig{0}_redlines.png".format(i))
    

In [ ]:
import matplotlib
matplotlib.colors.to_hex('tan', keep_alpha=False)